In [1]:
import sys
sys.path.append('../../')

In [2]:
preprocessed_dataset_path = 'preprocessed_dataset/'

In [3]:
from codes.docs.analysis import data_preprocessing, data_exploration, visualisation, data_preprocessing_high_dimension,model_training
from codes.docs.analysis.gene_analysis import genetic_file_preprocess

In [4]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from collections import defaultdict
%matplotlib inline
import tqdm

import statsmodels.api as sm
import statsmodels.formula.api as sfm

from scipy.stats import ttest_ind, pearsonr,zscore
import pandas as pd
import re

Here, the volumetric dataset is preprocessed to group the same brain regions.\
Then mass univariate test is preformed, across multiple regions.

# Main analysis

In [24]:
european_volumetric_dataset_term = pd.read_csv(
    preprocessed_dataset_path +
    'imaging_data/volumetric/term/european_volumetric_df_batch2.csv')

In [25]:
batch1_euro_ld = genetic_file_preprocess.Cohort.preprocess_PRSice_PRS_Anc_files(file_path='../../dataset/PRS/asd/ASD_eurold_lifted37_dHCP_merged_cleaned_EUROPEANS.gwas.all.score',column_prefix='batch1_euro_ld')
batch2_euro_ld = genetic_file_preprocess.Cohort.preprocess_PRSice_PRS_Anc_files(file_path='../../dataset/PRS/asd/ASD_nodups_euro_batch2_genotyped.gwas.all_score',column_prefix='batch2_euro_ld_no_dups')
batch2_euro_ld_imputed = genetic_file_preprocess.Cohort.preprocess_PRSice_PRS_Anc_files(file_path='../../dataset/PRS/asd/imputed_data/ASD_euro_batch2_imputed.gwas.all.score',column_prefix='batch2_euro_ld_imputed')

In [26]:
european_volumetric_dataset_term = european_volumetric_dataset_term.merge(batch2_euro_ld,left_on='ID',right_index=True)
european_volumetric_dataset_term = european_volumetric_dataset_term.merge(batch2_euro_ld_imputed,left_on='ID',right_index=True)

## AAL atlas

### performing individual volumes

#### European

In [31]:
df = european_volumetric_dataset_term.copy()
ancestry = [f'euro_Anc_PC{i}' for i in range(1,4)]
dependentVar = [
    i for i in european_volumetric_dataset_term.columns
    if bool(re.search('AAL \d',i))
]
df = df.rename(columns = {'AAL 93':'AAL 109'})
# df = df.rename(columns={i:i+'_ASD_All_LD' for i in df.columns if 'PRS' in i})


In [35]:
combined = df[[
    i for i in df.columns
    if bool(re.search('AAL \d', i))
]].copy()
# combined = df[[i for i in df.columns if 'imputed' in i]]
correlation_matrix = np.corrcoef(combined, rowvar=False)
_ = data_exploration.matSpDLite(correlation_matrix, alpha=0.05)

Effective Number of Independent Variables [Veff] is nan
Effective Number of Independent Variables [VeffLi] (Using equation 5 of Li and Ji 2005) is 0.0
The adjusted multiple testing correction p-val is alpha/lower(Meff) = nan


/Users/hai/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3723: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/hai/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/Users/hai/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [12]:
thresholds = [i for i in df.columns if 'PRS' in i]
dependentVar_cols=[i for i in df.columns if bool(re.search('AAL \d',i))]

european_volumetric_term_mass_univariate = data_exploration.MassUnivariate.calculate_mass_univariate_across_multiple_thresholds(
    df,
    thresholds=thresholds,
    cat_independentVar_cols=['Gender'],
    cont_independentVar_cols=['GA_vol', 'PMA_vol','17 ICV (all except bone)']+ancestry,
    dependentVar_cols=dependentVar_cols)

100%|██████████████████████████████████████████████████████| 11/11 [00:01<00:00,  5.76it/s]


In [13]:
european_volumetric_term_mass_univariate.sort_values(by='PRS_pval')

,Connection,const_coef,const_pval,GA_vol_coef,GA_vol_pval,PMA_vol_coef,PMA_vol_pval,17 ICV (all except bone)_coef,17 ICV (all except bone)_pval,euro_Anc_PC1_coef,euro_Anc_PC1_pval,euro_Anc_PC2_coef,euro_Anc_PC2_pval,euro_Anc_PC3_coef,euro_Anc_PC3_pval,PRS_coef,PRS_pval,Gender_2.0_coef,Gender_2.0_pval,threshold
933,AAL 4,4986.511239,2.608627e-226,-12.866497,0.701348,-6.928252,0.889239,757.212036,1.591010e-45,54.119598,0.045833,-28.892864,0.276845,10.734080,0.688304,95.131687,0.000413,-9.175244,0.868442,PRS_1_ASD_All_LD
840,AAL 4,4987.064190,3.481230e-226,-14.145631,0.673879,-3.438551,0.944946,754.887974,3.223481e-45,52.875507,0.051019,-28.914634,0.277706,11.687636,0.662596,92.365049,0.000602,-10.333133,0.852199,PRS_0.5_ASD_All_LD
626,AAL 69,1375.350756,3.999901e-175,9.185615,0.551865,-34.236205,0.134786,201.386766,7.080277e-21,3.689357,0.762280,-10.388588,0.388349,18.525387,0.131674,41.253482,0.000615,9.845640,0.699425,PRS_0.01_ASD_All_LD
654,AAL 4,4986.423213,6.200557e-226,-5.173498,0.877917,-0.759047,0.987860,752.804355,6.914859e-45,46.520424,0.083397,-22.622515,0.391607,14.887772,0.578722,86.904554,0.000994,-8.990915,0.871560,PRS_0.05_ASD_All_LD
378,AAL 7,8381.949834,6.828002e-242,-62.303360,0.192052,14.628937,0.836084,1315.182607,3.042636e-58,45.119470,0.232039,-32.418858,0.382412,73.219200,0.055640,124.846111,0.001102,-90.187687,0.265315,PRS_0.0001_ASD_All_LD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,AAL 56,4149.710051,2.343527e-242,16.226313,0.496554,-44.122625,0.214560,527.997714,5.019758e-44,-6.351274,0.738200,3.758150,0.839314,-46.544142,0.015482,-0.037353,0.998397,-65.055234,0.099101,PRS_1e-07_ASD_All_LD
302,AAL 24,3211.316080,6.266080e-233,8.374053,0.677389,16.682360,0.575107,431.217828,1.092973e-42,29.924563,0.060847,-12.099535,0.439040,29.710965,0.064092,0.031174,0.998406,-5.289014,0.875153,PRS_1e-05_ASD_All_LD
59,AAL 60,3057.423250,2.867355e-213,-24.586499,0.294158,102.199257,0.003679,428.618455,6.803769e-34,13.410668,0.470736,-3.001522,0.869230,-23.241448,0.216807,0.027902,0.998780,54.953451,0.155880,PRS_1e-08_ASD_All_LD
198,AAL 13,4936.524969,8.061358e-223,14.326481,0.677591,67.077392,0.189207,682.648168,2.495916e-38,37.338403,0.170777,-32.760371,0.221160,-5.405348,0.844851,-0.037736,0.998861,56.158184,0.322652,PRS_1e-06_ASD_All_LD


In [10]:
# prs_lds = ['All_LD','AllLDimputed','Euro_LD','EuroLDimputed']
prs_lds = ['All_LD']
for threshold in prs_lds:
    _,prs_pca,_ = data_preprocessing_high_dimension.FeatureReduction.perform_PCA(
    df = df,
    dependentVar_cols=[i for i in df.columns if threshold in i],
    scaling=True,
    n_components=1)
    for i in range(prs_pca.shape[1]):
        df[f'prs_pca_{threshold}_PC{i}'] = prs_pca[:,i]